# NFL Combine Drill Distributions

Beeswarm plots for all six combine drills — every recorded measurement from 2007 to 2026, colored by position group.

Uses `ggbeeswarm::geom_quasirandom` with Okabe-Ito colors (colorblind-safe) and `theme_spade` branding.

In [ ]:
library(ggplot2)
library(ggbeeswarm)
library(dplyr)
library(showtext)
library(arrow)
library(paletteer)

font_add_google("Karla", "karla")
showtext_auto()
showtext_opts(dpi = 300)

# Make inline plots large in the notebook
options(repr.plot.width = 20, repr.plot.height = 11)

theme_spade <- function(base_size = 12) {
  theme_minimal(base_size = base_size) +
    theme(
      text = element_text(family = "karla", color = "#333333"),
      plot.background = element_rect(fill = "#DDEBEC", color = NA),
      panel.background = element_rect(fill = "#DDEBEC", color = NA),
      plot.title = element_text(face = "bold", size = 36, margin = margin(b = 5)),
      plot.title.position = "plot",
      plot.subtitle = element_text(size = 22, color = "#666666", margin = margin(b = 15)),
      plot.caption = element_text(size = 18, color = "#888888", hjust = 0),
      plot.caption.position = "plot",
      panel.grid.major = element_line(color = "#cccccc", linewidth = 0.3),
      panel.grid.minor = element_blank(),
      legend.background = element_rect(fill = "#DDEBEC", color = NA),
      legend.key = element_rect(fill = "#DDEBEC", color = NA),
      plot.margin = margin(t = 20, r = 20, b = 20, l = 20)
    )
}

# Okabe-Ito (colorblind-safe) + extensions
oi <- paletteer_d("colorblindr::OkabeIto")
pos_colors <- c(
  "DB"  = oi[6],
  "WR"  = oi[2],
  "RB"  = oi[3],
  "LB"  = oi[5],
  "TE"  = oi[7],
  "FB"  = oi[4],
  "QB"  = oi[8],
  "P"   = oi[1],
  "DL"  = "#E83562",
  "LS"  = "#7A5195",
  "OL"  = "#1AFF1A"
)

output_dir <- file.path("..", "output", "beeswarm")
dir.create(output_dir, recursive = TRUE, showWarnings = FALSE)

In [ ]:
df <- read_parquet(file.path("..", "data", "combine_pro_day.parquet")) |>
  mutate(POS_GP = if_else(POS_GP %in% c("EDGE", "DT"), "DL", POS_GP)) |>
  filter(POS_GP %in% names(pos_colors)) |>
  mutate(POS_GP = factor(POS_GP, levels = names(pos_colors)))

cat(sprintf("Loaded %s rows (%d–%d)\n", format(nrow(df), big.mark = ","), min(df$Year), max(df$Year)))

In [ ]:
make_beeswarm <- function(col, title, xlabel, lo, hi, step, filename) {
  subset <- df |>
    filter(!is.na(.data[[col]]), .data[[col]] >= lo, .data[[col]] <= hi)
  n_obs <- nrow(subset)

  p <- ggplot(subset, aes(x = .data[[col]], y = 0, color = POS_GP)) +
    geom_quasirandom(
      groupOnX = TRUE, width = 0.4,
      size = 1, alpha = 0.6
    ) +
    scale_color_manual(values = pos_colors, name = NULL) +
    labs(
      title = title,
      subtitle = paste0("Every recorded measurement from 2007–2026 (n = ", format(n_obs, big.mark = ","), ")"),
      x = xlabel,
      y = NULL,
      caption = "Ray Carpenter | TheSpade.Substack.com | @csv_enjoyer | data: Various sources"
    ) +
    scale_x_continuous(breaks = seq(lo, hi, step)) +
    theme_spade() +
    theme(
      axis.text.x = element_text(size = 18),
      axis.title.x = element_text(size = 20),
      axis.text.y = element_blank(),
      panel.grid.major.y = element_blank(),
      legend.position = "top",
      legend.text = element_text(size = 16, face = "bold"),
      legend.key.size = unit(1.2, "lines")
    ) +
    guides(color = guide_legend(nrow = 1, override.aes = list(size = 4, alpha = 1)))

  out_path <- file.path(output_dir, filename)
  ggsave(out_path, plot = p, width = 16, height = 9, dpi = 300, bg = "#DDEBEC")
  cat(sprintf("Saved %s\n", out_path))
  p
}

In [ ]:
make_beeswarm(
  col = "40 Yard",
  title = "NFL Combine & Pro Day 40-Yard Dash Times by Position",
  xlabel = "40-Yard Dash (seconds)",
  lo = 4.2, hi = 5.5, step = 0.2,
  filename = "forty_spread.png"
)

In [ ]:
make_beeswarm(
  col = "Vert Leap (in)",
  title = "NFL Combine & Pro Day Vertical Leap by Position",
  xlabel = "Vertical Leap (inches)",
  lo = 20, hi = 46, step = 2,
  filename = "vert_leap_spread.png"
)

In [ ]:
make_beeswarm(
  col = "Broad Jump (in)",
  title = "NFL Combine & Pro Day Broad Jump by Position",
  xlabel = "Broad Jump (inches)",
  lo = 84, hi = 142, step = 4,
  filename = "broad_jump_spread.png"
)

In [ ]:
make_beeswarm(
  col = "3Cone",
  title = "NFL Combine & Pro Day 3-Cone Drill by Position",
  xlabel = "3-Cone Drill (seconds)",
  lo = 6.2, hi = 8.2, step = 0.2,
  filename = "three_cone_spread.png"
)

In [ ]:
make_beeswarm(
  col = "Shuttle",
  title = "NFL Combine & Pro Day 20-Yard Shuttle by Position",
  xlabel = "20-Yard Shuttle (seconds)",
  lo = 3.8, hi = 5.0, step = 0.2,
  filename = "shuttle_spread.png"
)

In [ ]:
make_beeswarm(
  col = "Bench Press",
  title = "NFL Combine & Pro Day Bench Press by Position",
  xlabel = "Bench Press (reps at 225 lbs)",
  lo = 5, hi = 50, step = 5,
  filename = "bench_press_spread.png"
)